### Day 3

Description of the puzzle [here](https://adventofcode.com/2019/day/3)

### Preliminaries

In [1]:
with open('day_3_input.txt', 'r') as f:
    data=[line.split(',') for line in f.readlines()]

### Part 1

#### Part 1 test

Test case

In [2]:
t1 = ['R8','U5','L5','D3']

In [3]:
t2 = ['U7','R6','D4','L4']

Function to create vectors of wire moves

**This is a good fail !**

**AKA: a good example of having misinterpreted the brief, but still having learned something.**

This was developed under the incorrect assumption that the task was to find intersections between wires only at node points in the paths, not at intermediate points along the paths.

In [4]:
def position_changes(t):
    tdx = np.zeros(len(t), dtype=int)
    tdy = np.zeros(len(t), dtype=int)
    for i, el in enumerate(t):
        if el[0] == 'R':
            tdx[i] += (int(el[1::])) 
        elif el[0] == 'L':
            tdx[i] -=  (int(el[1::]))        
        if el[0] == 'U':
            tdy[i] += (int(el[1::])) 
        elif el[0] == 'D':
            tdy[i] -= (int(el[1::]))    
    return list(zip(tdx, tdy))    

Function to create vectors of wire node coordinates 

From: https://stackoverflow.com/a/16425408/1034648

In [5]:
def accumulate_tuples(iterable):
    accum_a = accum_b = 0
    for a, b in iterable:
        accum_a += a
        accum_b += b
        yield accum_a, accum_b

In [6]:
coord_t1 = list(accumulate_tuples(moves_t1))
coord_t2 = list(accumulate_tuples(moves_t2))

coord_t1.insert(0, (0, 0))
coord_t2.insert(0, (0, 0))

NameError: name 'moves_t1' is not defined

In [ ]:
print(coord_t1)
print(coord_t2)

Find nodes at which wires intersect 

In [ ]:
[np.where(a == b) for a, b in zip (coord_t1, coord_t2)]

**That's a big fail above!!**

#### Part 2 solution

With this approach we similarly split each input into a direction and a number but now we turn it directly into a distance and we move that distance a square at a time, to create coordinates

In [ ]:
def positions(t):
    pos_list = []
    posx = 0
    posy = 0
    for el in t:
        direction = el[0]
        distance = int(el[1:])
        for d in range(distance):
            if direction == 'R':
                posx += 1               
            elif direction == 'L':
                posx -= 1    
            if direction == 'U':
                posy += 1 
            elif direction == 'D':
                posy -= 1 
            pos_list.append((posx, posy)) 
    return pos_list

In [ ]:
coords_t1 = positions(t1)
coords_t2 = positions(t2)

In [ ]:
coords_t1b

This works.
Now you have to find the intersections.
For that I should turn the output of the above into a set, because sets come with intersection

Then usce `scipy.spatial.distance.cdist` for distances

Also, move the if statements for directions outside of the for loop for efficiency